In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

In [14]:
# Inicializa navegador
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url = 'https://techguide.sh/pt-BR/path/java/'
driver.get(url)
time.sleep(3)  # Aguarda carregamento

# Títulos dos botões da página
expressoes_desejadas = []
pattern = re.compile(r'\*\*(.*?)\*\*')
with open('../nomes_botoes.txt', 'r') as arquivo:
    linhas = arquivo.readlines()
    for linha in linhas:
        
        match = pattern.search(linha)
        if match:
            expressoes_desejadas.append(match.group(1))

# Extrai HTML da página
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Localiza botões com <p> contendo as expressões

botoes_relevantes = []

for button in soup.find_all('button'):
    p_tag = button.find('p')
    if p_tag:
        texto = p_tag.get_text(strip=True)
        if any(expr.lower() in texto.lower() for expr in expressoes_desejadas):
            classes = button.get('class')
            if classes:
                botoes_relevantes.append({'texto': texto, 'classe': classes})  # Salva lista completa

In [ ]:
# Lista para armazenar links
todos_os_links = []

# Processa cada botão relevante
for botao in botoes_relevantes:
    try:
        condicoes_xpath = ' and '.join([f'contains(@class, "{c}")' for c in botao['classe']])
        xpath_botao = f'//button[{condicoes_xpath}]'

        elemento_botao = driver.find_element(By.XPATH, xpath_botao)
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", elemento_botao)
        time.sleep(0.5)
        elemento_botao.click()
        time.sleep(1)

        # Pega divs visíveis e captura links
        divs = driver.find_elements(By.TAG_NAME, 'div')
        divs_visiveis = [d for d in divs if d.is_displayed()]

        for div in divs_visiveis:
            links = div.find_elements(By.TAG_NAME, 'a')
            for link in links:
                href = link.get_attribute('href')
                text = link.text
                category = botao['texto']
            
                if href and href not in todos_os_links:
                    temp_dict = {}
                    temp_dict['categoria'] = category
                    temp_dict['texto'] = text
                    temp_dict['link'] = href 
                    todos_os_links.append(temp_dict)
                

    # Fecha a div
        close_icon = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/div[2]/div[1]/div[2]')
        close_icon.click()
        time.sleep(1)

    except Exception as e:
        print(f'Erro no botão "{botao["texto"]}": {e}')
        continue

print(f'Total de links capturados: {len(todos_os_links)}')
print(todos_os_links)

driver.quit()

Total de links capturados: 2766
[{'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://techguide.sh/'}, {'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://techguide.sh/pt-BR/path/java/#7days-of-code'}, {'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://techguide.sh/'}, {'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://techguide.sh/en-US/'}, {'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://techguide.sh/es/'}, {'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://techguide.sh/'}, {'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://techguide.sh/'}, {'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://techguide.sh/'}, {'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://7daysofcode.io/'}, {'categoria': 'Git e GitHub - Fundamentos', 'texto': '', 'link': 'https://alura.com.br/dev-em-t'}, {'catego

In [19]:
links_filtrado = [link for link in todos_os_links if link['texto'] != '']
print(len(todos_os_links))
print(len(links_filtrado))

2766
2042


In [67]:
from functions import get_source, create_soup

relacao_tipos = ['ARTIGO', 'CURSO', 'DESAFIO', 'LIVRO', 'PODCAST', 'SITE', 'YOUTUBE']



# Exportando csv

import pandas as pd
df = pd.DataFrame(links_filtrado)


# iterando para criar coluna tipo e limpar coluna texto
for index, linha in df.iterrows():
    if 'ARTIGO' in linha['texto']:
        df.at[index, 'texto'] = df.at[index, 'texto'].replace('ARTIGO', '')
        df.at[index, 'tipo'] = 'ARTIGO'
    if 'CURSO' in linha['texto']:
        df.at[index, 'texto'] = df.at[index, 'texto'].replace('CURSO', '')
        df.at[index, 'tipo'] = 'CURSO'
        
        soup = create_soup(linha['link'])
        paragrafos = soup.find_all('p')
        padrao = re.compile(r'\d+h')
        for p in paragrafos:
            match = padrao.search(p.text)
            if match:
                df.at[index, 'carga_horaria'] = match
        
    if 'DESAFIO' in linha['texto']:
        df.at[index, 'texto'] = df.at[index, 'texto'].replace('DESAFIO', '')
        df.at[index, 'tipo'] = 'DESAFIO'
    if 'LIVRO' in linha['texto']:
        df.at[index, 'texto'] = df.at[index, 'texto'].replace('LIVRO', '')
        df.at[index, 'tipo'] = 'LIVRO'
    if 'PODCAST' in linha['texto']:
        df.at[index, 'texto'] = df.at[index, 'texto'].replace('PODCAST', '')
        df.at[index, 'tipo'] = 'PODCAST'
    if 'YOUTUBE' in linha['texto']:
        df.at[index, 'texto'] = df.at[index, 'texto'].replace('YOUTUBE', '')
        df.at[index, 'tipo'] = 'YOUTUBE'
    if 'SITE' in linha['texto']:
        df.at[index, 'texto'] = df.at[index, 'texto'].replace('SITE', '')
        df.at[index, 'tipo'] = 'SITE'



# Continuar criando colunas de tipo (SITE, ARTIGO, CURSO etc) e carga horaria dos cursos



df.to_csv('links_filtrado.csv')

In [66]:
for index, linha in df.iterrows():
    if linha['tipo'] == 'CURSO':
        print(linha['texto'])
        #soup = create_soup(linha['link'])
        #paragrafos = soup.find_all('p')
        #print(p.text)

Curso Git e GitHub: compartilhando e colaborando em projetos
Curso Git e GitHub: dominando controle de versão de código
Curso Git e GitHub: compartilhando e colaborando em projetos
Curso Git e GitHub: dominando controle de versão de código
Curso Git e GitHub: compartilhando e colaborando em projetos
Curso Git e GitHub: dominando controle de versão de código
Curso Git e GitHub: compartilhando e colaborando em projetos
Curso Git e GitHub: dominando controle de versão de código
Curso Git e GitHub: compartilhando e colaborando em projetos
Curso Git e GitHub: dominando controle de versão de código
Curso Git e GitHub: compartilhando e colaborando em projetos
Curso Git e GitHub: dominando controle de versão de código
Curso HTTP: Entendendo a web por baixo dos panos
Curso HTTP: Entendendo a web por baixo dos panos
Curso HTTP: Entendendo a web por baixo dos panos
Curso HTTP: Entendendo a web por baixo dos panos
Curso HTTP: Entendendo a web por baixo dos panos
Curso HTTP: Entendendo a web por ba

In [61]:
df

,categoria,texto,link,tipo,carga_horaria
0,Git e GitHub - Fundamentos,GitHub Documentação,https://docs.github.com/pt,SITE,NaN
1,Git e GitHub - Fundamentos,GitHub Pages Documentação,https://docs.github.com/pt/pages/getting-start...,SITE,NaN
2,Git e GitHub - Fundamentos,Git School - Visualizing Git,https://git-school.github.io/visualizing-git/,SITE,NaN
3,Git e GitHub - Fundamentos,"Dangit, Git!?!",https://dangitgit.com/,SITE,NaN
4,Git e GitHub - Fundamentos,Rafaella Ballerini: O que é Git e GitHub? - de...,https://www.youtube.com/watch?v=DqTITcMq68k,YOUTUBE,NaN
...,...,...,...,...,...
2037,Conceitos de Design Orientado a Domínio (Domai...,Domain-Driven Design (DDD) - O que é?,https://cursos.alura.com.br/extra/alura-mais/d...,SITE,NaN
2038,Conceitos de Design Orientado a Domínio (Domai...,Primeiras aulas do curso Java e Domain Driven ...,https://www.alura.com.br/conteudo/java-domain-...,SITE,NaN
2039,Conceitos de Design Orientado a Domínio (Domai...,Curso Java e Domain Driven Design: apresentand...,https://www.alura.com.br/curso-online-java-dom...,CURSO,"<re.Match object; span=(72, 75), match='24h'>"
2040,Conceitos de Design Orientado a Domínio (Domai...,Primeiras aulas do curso PHP e Domain Driven D...,https://www.alura.com.br/conteudo/domain-drive...,SITE,NaN


In [57]:
df.to_csv('links_filtrado.csv')